# Recipe Recommender System

**Want a specific meal for within a certain budget for a specific number of people**

In [1]:
import pandas as pd
import numpy as np
import pickle

## Read in Files

### Grocery Similarity Model

In [2]:
grocery_similarities = pd.read_csv('../../data/05_model_output/data_matrix_sim.csv')

In [3]:
grocery_similarities.set_index('Unnamed: 0', inplace=True)

Let's take a look at our cosine similarity matrix.

In [4]:
grocery_similarities

,acorn squash,aduki beans,agave,albacore,alfalfa sprouts,alfredo,all,allspice,almond,almond butter,...,yahrzeit,yam,yamaimo,yan,yasai,year,yogurt kefir,yolk,zing,zinger babies
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
acorn squash,1.000000,0.001239,0.001198,0.001979,0.002560,0.000000,0.004003,0.002764,0.011173,0.007193,...,0.000000,0.003787,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
aduki beans,0.001239,1.000000,0.000000,0.000000,0.000000,0.000000,0.001002,0.000000,0.001427,0.002701,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
agave,0.001198,0.000000,1.000000,0.002396,0.000930,0.000910,0.000969,0.000000,0.009386,0.007838,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
albacore,0.001979,0.000000,0.002396,1.000000,0.000000,0.000000,0.010672,0.000000,0.005319,0.002877,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
alfalfa sprouts,0.002560,0.000000,0.000930,0.000000,1.000000,0.000778,0.002070,0.000000,0.006722,0.004783,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
alfredo,0.000000,0.000000,0.000910,0.000000,0.000778,1.000000,0.001622,0.000000,0.002887,0.002342,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
all,0.004003,0.001002,0.000969,0.010672,0.002070,0.001622,1.000000,0.003353,0.015676,0.007647,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
allspice,0.002764,0.000000,0.000000,0.000000,0.000000,0.000000,0.003353,1.000000,0.002388,0.005165,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.015530,0.000000
almond,0.011173,0.001427,0.009386,0.005319,0.006722,0.002887,0.015676,0.002388,1.000000,0.045738,...,0.000000,0.000873,0.000718,0.001708,0.000000,0.000000,0.0,0.000000,0.001708,0.001871


Because this was an item-item based similarity the output gives us items that are most like the other items. That means that when I put in parsley I get out other seasonings like the list shown below. In order to get things that go with ingredients we will need do do this by user-item similarity instead. Let's run this later. 

In [5]:
print(grocery_similarities.loc['parsley'].nlargest(11))

parsley     1.000000
garlic      0.161545
onion       0.159124
cilantro    0.153014
lemon       0.151143
tomatoes    0.128387
thyme       0.121335
dill        0.119339
onions      0.118740
small       0.117934
kale        0.117458
Name: parsley, dtype: float64


### Recipe Database

In [6]:
infile = open('../../data/03_processed/crf_ingred_dict','rb')
recipe_dict = pickle.load(infile)
infile.close()

In [7]:
infile = open('../../data/03_processed/crf_links_dict','rb')
recipe_links_dict = pickle.load(infile)
infile.close()

In [8]:
recipe_dict['apple carrot cupcakes']

[{'qty': '2',
  'unit': 'cups',
  'name': '260 g all purpose flour',
  'comment': ''},
 {'qty': '1', 'unit': '', 'name': '1/4 ', 'comment': '250 g cups sugar'},
 {'qty': '2', 'unit': 'teaspoons', 'name': 'baking soda', 'comment': ''},
 {'qty': '1',
  'unit': '1/2  teaspoons',
  'name': 'baking powder',
  'comment': ''},
 {'qty': '1', 'unit': '1/2  teaspoons', 'name': 'cinnamon', 'comment': ''},
 {'qty': '1/4 ', 'unit': 'teaspoon', 'name': 'ground nutmeg', 'comment': ''},
 {'qty': '1/8 ', 'unit': 'teaspoon', 'name': 'ground cloves', 'comment': ''},
 {'qty': '1/4 ', 'unit': 'teaspoon', 'name': 'salt', 'comment': ''},
 {'qty': '1',
  'unit': 'cup',
  'name': '236 ml vegetable oil',
  'comment': 'light olive oil or canola oil'},
 {'qty': '4', 'unit': '', 'name': 'large eggs', 'comment': 'lightly beaten'},
 {'qty': '1',
  'unit': '1/2  cups',
  'name': 'packed coarsely grated carrots',
  'comment': 'about 3 medium'},
 {'qty': '1',
  'unit': '1/2  cups',
  'name': 'packed coarsely grated tar

In [9]:
recipe_links_dict['apple carrot cupcakes']

'https://www.simplyrecipes.com/recipes/apple_carrot_cupcake/'

### Grocery Prices 

In [10]:
grocery_prices_marianos = pd.read_csv('../../data/03_processed/grocery_prices_marianos_final.csv')

In [11]:
grocery_prices_marianos

,product,main_price,prod_aile,price_per_lb,measure_words_main_price,item_weight_count_vol,date_collected,store,location,product_new
0,Green Onions (Scallions),$0.99,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,scallions
1,Cilantro,$0.79,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,cilantro
2,Italian Parsley,$0.99,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,parsley
3,Roundy's Organic Fresh Rosemary,$2.50,Fresh Herbs,NaN,each,0.75 ounce,2019-08-28,Marianos,60615,roundys
4,Roundy's Organic Fresh Thyme,$2.50,Fresh Herbs,NaN,each,0.75 ounce,2019-08-28,Marianos,60615,thyme
5,Roundy's Mint,$2.50,Fresh Herbs,NaN,each,0.75 ounce,2019-08-28,Marianos,60615,mint
6,Basil,$1.79,Fresh Herbs,NaN,each,1 each,2019-08-28,Marianos,60615,basil
7,Dill,$1.99,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,dill
8,Roundy's Organics Fresh Dill,$2.50,Fresh Herbs,NaN,each,1 bunch,2019-08-28,Marianos,60615,dill
9,Gourmet Garden™ Ginger Stir-in Paste,$4.49,Fresh Herbs,NaN,each,4 ounce,2019-08-28,Marianos,60615,paste


***

In [12]:
def extract_ingredients(single_recipe_ing_list):
    ret = []
    for ingredient in single_recipe_ing_list:
        ret.append(ingredient['name'])
    return ret 

In [13]:
def find_recipe_by_ingredient(recipe_dict, search_item):
    
    listofkeys = []
    listofitems = recipe_dict.items()

    for item in listofitems:
        lst = extract_ingredients(item[1])
        subs = search_item
        if any(subs in s for s in lst):
            listofkeys.append(item[0])
    return listofkeys

In [14]:
print(grocery_similarities.loc['parsley'].nlargest(11))

parsley     1.000000
garlic      0.161545
onion       0.159124
cilantro    0.153014
lemon       0.151143
tomatoes    0.128387
thyme       0.121335
dill        0.119339
onions      0.118740
small       0.117934
kale        0.117458
Name: parsley, dtype: float64


In [15]:
find_recipe_by_ingredient(recipe_dict, 'chocolate')

['easy chocolate pudding in the microwave',
 'paleo hot cocoa',
 'cincinnati chili',
 'classic ice cream sandwiches',
 'red velvet cookies',
 'peanut butter pretzel magic bars',
 'peanut butter chocolate chip banana bread',
 'peanut butter blossoms',
 'oatmeal almond butter breakfast cookies',
 'thin and crispy chocolate chip cookies',
 'peanut butter swirl brownies',
 'brookies (chocolate chip brownie cookies)',
 'double chocolate cupcakes',
 'german chocolate snack cake with coconut-pecan frosting',
 's’mores cookies',
 'fudgy chocolate brownies',
 'chocolate banana bread',
 'chocolate bundt cake',
 'chocolate peanut butter bars',
 'walnut mocha torte',
 'white chocolate macadamia nut cookies',
 'cowboy cookies',
 'carmelitas',
 'peppermint bark',
 'chocolate ganache torte',
 'christmas crack',
 '7-layer magic bars']

In [16]:
recipe_dict['cincinnati chili']

[{'qty': '2',
  'unit': 'pounds',
  'name': 'ground beef',
  'comment': '80 20 is good'},
 {'qty': '1', 'unit': '6 ounce', 'name': 'can tomato paste', 'comment': ''},
 {'qty': '4', 'unit': 'cups', 'name': 'water', 'comment': ''},
 {'qty': '1', 'unit': '8 ounce', 'name': 'can tomato sauce', 'comment': ''},
 {'qty': '1',
  'unit': '',
  'name': 'large onion',
  'comment': 'minced about 3 cups'},
 {'qty': '6', 'unit': 'cloves', 'name': 'garlic', 'comment': 'minced'},
 {'qty': '3', 'unit': 'tablespoons', 'name': 'chili powder', 'comment': ''},
 {'qty': '1', 'unit': 'teaspoon', 'name': 'cumin', 'comment': ''},
 {'qty': '1', 'unit': 'teaspoon', 'name': 'cinnamon', 'comment': ''},
 {'qty': '3/4 ', 'unit': 'teaspoon', 'name': 'ground allspice', 'comment': ''},
 {'qty': '1/4 ', 'unit': 'teaspoon', 'name': 'ground cloves', 'comment': ''},
 {'qty': '1/2 ', 'unit': 'teaspoon', 'name': 'cayenne', 'comment': ''},
 {'qty': '2', 'unit': 'teaspoons', 'name': 'kosher salt', 'comment': ''},
 {'qty': '2',

## Recommendor System sudo-code

```python


```